## Step 1 : Scraping and storing data

In [25]:
import csv
import requests
from bs4 import BeautifulSoup

# List of website URLs
urls = [
    "https://www.aljazeera.net/encyclopedia/2016/6/6/%D8%A7%D9%84%D8%B0%D9%83%D8%A7%D8%A1-%D8%A7%D9%84%D8%A7%D8%B5%D8%B7%D9%86%D8%A7%D8%B9%D9%8A-%D8%AD%D9%8A%D9%86%D9%85%D8%A7-%D8%AA%D9%81%D9%83%D8%B1-%D8%A7%D9%84%D8%A2%D9%84%D8%A9",
    "https://www.oracle.com/ae-ar/artificial-intelligence/what-is-ai/",
    "https://aws.amazon.com/ar/what-is/artificial-intelligence/"
]

# Function to scrape paragraphs from a URL
def scrape_paragraphs_from_url(url):
    try:
        # Send HTTP request
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for invalid responses

        # Parse HTML content
        soup = BeautifulSoup(response.content, "html.parser")

        # Extract paragraphs
        paragraphs = [p.get_text().strip() for p in soup.find_all("p") if p.get_text().strip()]

        return paragraphs
    except Exception as e:
        print(f"Error occurred while scraping {url}: {e}")
        return None

# Main function to scrape paragraphs from all URLs
def scrape_paragraphs_from_all_urls(urls):
    all_paragraphs = []
    for url in urls:
        paragraphs = scrape_paragraphs_from_url(url)
        if paragraphs:
            all_paragraphs.extend(paragraphs)
    return all_paragraphs

# Calling the main function to scrape paragraphs from all URLs
all_paragraphs = scrape_paragraphs_from_all_urls(urls)

# Write paragraphs to a CSV file (without scores)
with open("paragraphs.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    for paragraph in all_paragraphs:
        if paragraph:
            writer.writerow([paragraph])


## Step 2: Calculate Relevance Scores with TF-IDF

1. We import TfidfVectorizer from sklearn.feature_extraction.text to calculate TF-IDF scores.
2. After scraping paragraphs from all URLs, we calculate TF-IDF scores for each paragraph using TfidfVectorizer.
3. Finally, we write the paragraphs along with their TF-IDF scores to a CSV file. Each row contains a paragraph and its corresponding TF-IDF score.

In [26]:
import csv
import requests
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer

# List of website URLs
urls = [
    "https://www.aljazeera.net/encyclopedia/2016/6/6/%D8%A7%D9%84%D8%B0%D9%83%D8%A7%D8%A1-%D8%A7%D9%84%D8%A7%D8%B5%D8%B7%D9%86%D8%A7%D8%B9%D9%8A-%D8%AD%D9%8A%D9%86%D9%85%D8%A7-%D8%AA%D9%81%D9%83%D8%B1-%D8%A7%D9%84%D8%A2%D9%84%D8%A9",
    "https://www.oracle.com/ae-ar/artificial-intelligence/what-is-ai/",
    "https://aws.amazon.com/ar/what-is/artificial-intelligence/"
]

# Function to scrape paragraphs from a URL
def scrape_paragraphs_from_url(url):
    try:
        # Send HTTP request
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for invalid responses

        # Parse HTML content
        soup = BeautifulSoup(response.content, "html.parser")

        # Extract paragraphs
        paragraphs = [p.get_text().strip() for p in soup.find_all("p") if p.get_text().strip()]

        return paragraphs
    except Exception as e:
        print(f"Error occurred while scraping {url}: {e}")
        return None

# Main function to scrape paragraphs from all URLs
def scrape_paragraphs_from_all_urls(urls):
    all_paragraphs = []
    for url in urls:
        paragraphs = scrape_paragraphs_from_url(url)
        if paragraphs:
            all_paragraphs.extend(paragraphs)
    return all_paragraphs

# Call the main function to scrape paragraphs from all URLs
all_paragraphs = scrape_paragraphs_from_all_urls(urls)

# Calculate TF-IDF scores
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_paragraphs)

# Write paragraphs with TF-IDF scores to a CSV file
with open("paragraphs_with_scores.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Paragraph", "TF-IDF Score"])
    for i, paragraph in enumerate(all_paragraphs):
        if paragraph:
            tfidf_score = tfidf_matrix[i].sum()
            writer.writerow([paragraph, tfidf_score])


## Step 3 : Preprocessing NLP Pipeline

In [27]:
import nltk
import csv
import re
import qalsadi.lemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    # Discretization
    discretized_text = re.sub(r'\d+', '<رقم>', text)

    # Text Cleaning
    cleaned_text = re.sub(r'[^\w\s]', '', discretized_text)
    cleaned_text = cleaned_text.lower()

    # Tokenization
    tokens = word_tokenize(cleaned_text)

    # Stop Words Removal
    stop_words = set(stopwords.words('arabic'))
    filtered_tokens = [token for token in tokens if token not in stop_words]

    # Lemmatization using qalsadi library
    lemmer = qalsadi.lemmatizer.Lemmatizer()
    lemmas = lemmer.lemmatize_text(' '.join(filtered_tokens))

    return ' '.join(lemmas)  # Return as a single string

# Read data from the CSV file
data_from_csv = []
with open("paragraphs_with_scores.csv", mode="r", encoding="utf-8") as file:
    reader = csv.reader(file)
    for row in reader:
        data_from_csv.append(row)  

# Preprocess the data
preprocessed_data = [preprocess_text(paragraph[0]) for paragraph in data_from_csv[1:]]

# Adding preprocessed data to existing data
for i, paragraph in enumerate(preprocessed_data):
    data_from_csv[i + 1].append(paragraph)

# Writting combined data to a new CSV file
with open("paragraphs_with_scores.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerows(data_from_csv)

# Printing a sample of the preprocessed data to verify
for i, lemmas in enumerate(preprocessed_data[:5]):
    print(f"Sample {i+1}: {lemmas}")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Sample 1: ذكاء اصطناع تعبير طلق قدر تبدي آلة برامج حاكى قدر ذهن بشر تعلم استنتاج فعل أوضاع برمج آلة أن اسم حقل أكاديمي معني كيفية صنع حواسيب برامج قادر اتخاذ سلوك ذكي
Sample 2: سم عصر حديث بارز طور إنس خدم إنجاز مهمة دقة كبير سرع عال موثوق رصين
Sample 3: رافق إنجاز تحدي اقتصاد سب جو هائل توزيع ثروة دول متقدم نامة بين أفراد نفس بلد واحد صار تهديد كبير عدد وظائف
Sample 4: عاد طرح مصطلح ذكاء اصطناع عالم حاسوب الأميركي جون مكارثي رقمرقم صاغ عام رقم عام شهد انعقاد مؤتمر علم كلية دارتموث الأميركية إشارة أبحاث جار آنذاك حول إمكان تصميم آلة ذكي قادر تقليد محاكاة عمل بشر
Sample 5: تم إعلان مؤتمر حمل جمع تبرع مال دعم أبحاث وصول اختراع أشبه عقل بشر يم آلة عمل مفرد حاج إنس


# Step 3: Train Models


In [12]:
import csv
# we have our preprocessed_text and labels (target) ready
text_data = []
labels = []
with open("paragraphs_with_scores.csv", mode="r", encoding="utf-8") as file:
    reader = csv.DictReader(file)
    for row in reader:
        text = row["text"]
        label = float(row["TF-IDF Score"])  # TF-IDF score as label
        text_data.append(text)
        labels.append(label)

print(labels)

[5.8514540358342435, 4.164563147722379, 5.010368571788386, 6.258658968009476, 5.051413539726722, 5.439024235762571, 5.449427317191826, 5.888641606110186, 5.655707819239928, 4.8791732950371145, 6.516580708318667, 4.546160802324115, 6.4317610186423115, 5.526665424433725, 4.330748326261495, 5.068805909846542, 5.259708281951149, 5.171544225953361, 5.658538210611621, 5.797123116403549, 4.612830843280806, 5.61885802533116, 5.47879701554365, 4.545090547601191, 5.330211732393721, 4.900260052401799, 4.160820389123455, 6.189793118159144, 5.358373765410045, 5.388021054506444, 5.215969435930708, 4.708777830647453, 6.252724519525624, 5.2570521255412554, 5.6994557605197995, 6.0256879998796835, 6.257619160661595, 5.209969063281068, 6.811319023445039, 4.941736432417115, 5.098650412960419, 5.779025500126466, 6.309086769561074, 5.707415436257148, 1.888402711259873, 2.917805048541531, 1.4513855654334076, 6.190674906791254, 6.098700841461703, 4.806704334136747, 5.436218224019627, 6.839042310212961, 6.6855

In [28]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import csv
import numpy as np

# Loading and preprocessing the data
text_data = []
scores = []

with open("paragraphs_with_scores.csv", mode="r", encoding="utf-8") as file:
    reader = csv.DictReader(file)
    for row in reader:
        text = row["text"]
        score = float(row["TF-IDF Score"])
        text_data.append(text)
        scores.append(score)

# Tokenization and padding
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(text_data)
sequences = tokenizer.texts_to_sequences(text_data)
padded_sequences = pad_sequences(sequences, maxlen=100)  # Maxlen is an example, adjust based on our data

# Assuming 'label' is the column containing the labels
X = padded_sequences
y = labels 

# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

### 1. Simple RNN Model

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

def create_rnn_model(vocab_size):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=128),
        SimpleRNN(128, return_sequences=False),
        Dense(1)  # For regression, no activation function here
    ])
    return model

# Create and compile the model
vocab_size = len(tokenizer.word_index) + 1
rnn_model = create_rnn_model(vocab_size)
rnn_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
rnn_history = rnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))
rnn_model.save("rnn_model.keras")

# Evaluate the model
rnn_eval = rnn_model.evaluate(X_val, y_val)
print(f"RNN Model Evaluation - Loss: {rnn_eval[0]}, MAE: {rnn_eval[1]}")


Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 172ms/step - loss: 25.7185 - mae: 4.7835 - val_loss: 11.2923 - val_mae: 3.0560
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 7.9849 - mae: 2.5379 - val_loss: 3.6120 - val_mae: 1.6474
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - loss: 2.6770 - mae: 1.3860 - val_loss: 2.2183 - val_mae: 1.1984
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 2.1553 - mae: 1.0705 - val_loss: 2.4566 - val_mae: 1.1850
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 2.5330 - mae: 1.1956 - val_loss: 2.3997 - val_mae: 1.1815
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 2.2311 - mae: 1.0976 - val_loss: 2.2060 - val_mae: 1.1904
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 1.7050 - mae: 0.9454 - val_loss: 2.3674 - val_mae: 1.2668
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 2.3727 - mae: 1.2080 - val_loss: 2.6115 - val_mae: 1.3475
Epoch 9/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 1.9318 - mae: 1.130

### - Manual Hyperparameter Tuning for RNN

In [32]:
from tensorflow.keras.optimizers import Adam

def create_rnn_model(vocab_size, rnn_units):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=128),
        SimpleRNN(rnn_units, return_sequences=False),
        Dense(1)  # For regression
    ])
    return model

# Define a list of hyperparameters to test
rnn_units_list = [64, 128, 256]
learning_rates = [0.001, 0.0001]
batch_sizes = [16, 32, 64]
epochs = 20  

best_model = None
best_val_mae = float('inf')
best_hyperparams = {}

for rnn_units in rnn_units_list:
    for lr in learning_rates:
        for batch_size in batch_sizes:
            print(f"Training model with {rnn_units} units, learning rate {lr}, batch size {batch_size}")
            
            # Create and compile the model
            rnn_model = create_rnn_model(vocab_size, rnn_units)
            optimizer = Adam(learning_rate=lr)
            rnn_model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])
            
            # Train the model
            rnn_history = rnn_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val), verbose=0)
            
            # Evaluate the model
            val_loss, val_mae = rnn_model.evaluate(X_val, y_val, verbose=0)
            print(f"Validation MAE: {val_mae}")
            
            # Save the best model
            if val_mae < best_val_mae:
                best_val_mae = val_mae
                best_model = rnn_model
                best_hyperparams = {'rnn_units': rnn_units, 'learning_rate': lr, 'batch_size': batch_size}
                best_model.save("best_rnn_model.keras")

print(f"Best Validation MAE: {best_val_mae}")
print(f"Best Hyperparameters: {best_hyperparams}")


Training model with 64 units, learning rate 0.001, batch size 16
Validation MAE: 1.4553802013397217
Training model with 64 units, learning rate 0.001, batch size 32
Validation MAE: 1.213857889175415
Training model with 64 units, learning rate 0.001, batch size 64
Validation MAE: 1.1840791702270508
Training model with 64 units, learning rate 0.0001, batch size 16
Validation MAE: 1.550737977027893
Training model with 64 units, learning rate 0.0001, batch size 32
Validation MAE: 3.0280940532684326
Training model with 64 units, learning rate 0.0001, batch size 64
Validation MAE: 4.217204570770264
Training model with 128 units, learning rate 0.001, batch size 16
Validation MAE: 1.2640485763549805
Training model with 128 units, learning rate 0.001, batch size 32
Validation MAE: 1.3441420793533325
Training model with 128 units, learning rate 0.001, batch size 64
Validation MAE: 1.2044482231140137
Training model with 128 units, learning rate 0.0001, batch size 16
Validation MAE: 1.737788677215

* Number of RNN units: Experiment with different numbers of units in the RNN layer.
* Learning rate: Adjust the learning rate of the optimizer.
* Batch size: Test different batch sizes.
* Number of epochs: Increase or decrease the number of epochs to see the effect on performance.

° Explanation
*   rnn_units_list: A list of different numbers of units to try in the RNN layer.
*   learning_rates: A list of different learning rates to test.
*   batch_sizes: A list of different batch sizes to test.
*   epochs: Set to 20 for more thorough training.
*   The code loops through all combinations of these hyperparameters, trains a model for each combination, and evaluates its performance on the validation set. 
*   The model with the best validation MAE (Mean Absolute Error) is saved as the best model, and its hyperparameters are recorded.

### 2. Bidirectional RNN Model

In [35]:
from tensorflow.keras.layers import Bidirectional

def create_birnn_model(vocab_size):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=128),
        Bidirectional(SimpleRNN(128, return_sequences=False)),
        Dense(1)  # For regression
    ])
    return model

# Create and compile the model
birnn_model = create_birnn_model(vocab_size)
birnn_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
birnn_history = birnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))
birnn_model.save("birnn_model.keras")

# Evaluate the model
birnn_eval = birnn_model.evaluate(X_val, y_val)
print(f"Bidirectional RNN Model Evaluation - Loss: {birnn_eval[0]}, MAE: {birnn_eval[1]}")


Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 252ms/step - loss: 26.1260 - mae: 4.8838 - val_loss: 12.9269 - val_mae: 3.2641
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 9.6302 - mae: 2.7285 - val_loss: 4.7760 - val_mae: 1.8417
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 2.3557 - mae: 1.2105 - val_loss: 2.6533 - val_mae: 1.3740
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 2.2855 - mae: 1.2025 - val_loss: 3.0673 - val_mae: 1.4337
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 1.7490 - mae: 0.9725 - val_loss: 3.1874 - val_mae: 1.5598
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 2.1545 - mae: 1.2054 - val_loss: 1.8090 - val_mae: 1.0866
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 0.9904 - mae: 0.8049 - val_loss: 2.9480 - val_mae: 1.4662
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 0.9756 - mae: 0.7928 - val_loss: 1.4236 - val_mae: 0.9592
Epoch 9/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 0.6716 - mae: 0.6054

### 3. GRU Model


In [34]:
from tensorflow.keras.layers import GRU

def create_gru_model(vocab_size):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=128),
        GRU(128, return_sequences=False),
        Dense(1)  # For regression
    ])
    return model

# Create and compile the model
gru_model = create_gru_model(vocab_size)
gru_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
gru_history = gru_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))
gru_model.save("gru_model.keras")

# Evaluate the model
gru_eval = gru_model.evaluate(X_val, y_val)
print(f"GRU Model Evaluation - Loss: {gru_eval[0]}, MAE: {gru_eval[1]}")


Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 412ms/step - loss: 28.4219 - mae: 5.1368 - val_loss: 28.9050 - val_mae: 5.1631
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 292ms/step - loss: 27.2981 - mae: 5.0026 - val_loss: 26.6421 - val_mae: 4.9344
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 164ms/step - loss: 23.9059 - mae: 4.6668 - val_loss: 21.2007 - val_mae: 4.3072
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 136ms/step - loss: 15.3753 - mae: 3.6368 - val_loss: 5.6694 - val_mae: 1.7982
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 139ms/step - loss: 5.9664 - mae: 1.7358 - val_loss: 4.6723 - val_mae: 1.8898
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 163ms/step - loss: 2.8121 - mae: 1.4490 - val_loss: 2.9037 - val_mae: 1.4174
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 164ms/step - loss: 1.0379 - mae: 0.8007 - val_loss: 1.0409 - val_mae: 0.7920
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 131ms/step - loss: 1.4921 - mae: 1.0815 - val_loss: 1.2227 - val_mae: 0.8730
Epoch 9/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 136ms/step - loss: 0.6359 

### 4. LSTM Model

In [33]:
from tensorflow.keras.layers import LSTM

def create_lstm_model(vocab_size):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=128),
        LSTM(128, return_sequences=False),
        Dense(1)  # For regression
    ])
    return model

# Create and compile the model
lstm_model = create_lstm_model(vocab_size)
lstm_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
# Best Hyperparameters: {'model_type': 'lstm', 'rnn_units': 64, 'learning_rate': 0.001, 'batch_size': 16}
# Train the model
lstm_history = lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))
lstm_model.save("lstm_model.keras")

# Evaluate the model
lstm_eval = lstm_model.evaluate(X_val, y_val)
print(f"LSTM Model Evaluation - Loss: {lstm_eval[0]}, MAE: {lstm_eval[1]}")


Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 278ms/step - loss: 27.9732 - mae: 5.0711 - val_loss: 27.2851 - val_mae: 4.9542
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - loss: 22.7782 - mae: 4.4448 - val_loss: 4.7139 - val_mae: 1.8912
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 187ms/step - loss: 2.8704 - mae: 1.3681 - val_loss: 3.3043 - val_mae: 1.3281
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 183ms/step - loss: 2.9256 - mae: 1.2795 - val_loss: 2.3226 - val_mae: 1.1836
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 125ms/step - loss: 2.1725 - mae: 1.0878 - val_loss: 2.5138 - val_mae: 1.3096
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 179ms/step - loss: 2.3391 - mae: 1.2498 - val_loss: 2.7387 - val_mae: 1.3886
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 126ms/step - loss: 2.4988 - mae: 1.2933 - val_loss: 2.4202 - val_mae: 1.2825
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 209ms/step - loss: 2.0209 - mae: 1.1144 - val_loss: 2.1780 - val_mae: 1.1926
Epoch 9/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 188ms/step - loss: 2.0711 - ma

### ° Hyperparameter Tuning for Bidirectional RNN, GRU, and LSTM

In [36]:
from tensorflow.keras.layers import Bidirectional, LSTM, GRU

def create_bidirectional_rnn_model(vocab_size, rnn_units):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=128),
        Bidirectional(SimpleRNN(rnn_units, return_sequences=False)),
        Dense(1)  
    ])
    return model

def create_gru_model(vocab_size, rnn_units):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=128),
        GRU(rnn_units, return_sequences=False),
        Dense(1)  
    ])
    return model

def create_lstm_model(vocab_size, rnn_units):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=128),
        LSTM(rnn_units, return_sequences=False),
        Dense(1)  
    ])
    return model

# Defining a list of hyperparameters to test
rnn_units_list = [64, 128, 256]
learning_rates = [0.001, 0.0001]
batch_sizes = [16, 32, 64]
epochs = 20  # we can increase the number of epochs for better tuning

best_model = None
best_val_mae = float('inf')
best_hyperparams = {}

for model_type in ['bidirectional_rnn', 'gru', 'lstm']:
    for rnn_units in rnn_units_list:
        for lr in learning_rates:
            for batch_size in batch_sizes:
                print(f"Training {model_type} model with {rnn_units} units, learning rate {lr}, batch size {batch_size}")
                
                if model_type == 'bidirectional_rnn':
                    model = create_bidirectional_rnn_model(vocab_size, rnn_units)
                elif model_type == 'gru':
                    model = create_gru_model(vocab_size, rnn_units)
                elif model_type == 'lstm':
                    model = create_lstm_model(vocab_size, rnn_units)
                    
                optimizer = Adam(learning_rate=lr)
                model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])
                
                history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val), verbose=0)
                
                val_loss, val_mae = model.evaluate(X_val, y_val, verbose=0)
                print(f"Validation MAE: {val_mae}")
                
                if val_mae < best_val_mae:
                    best_val_mae = val_mae
                    best_model = model
                    best_hyperparams = {'model_type': model_type, 'rnn_units': rnn_units, 'learning_rate': lr, 'batch_size': batch_size}
                    best_model.save(f"best_{model_type}_model.keras")

print(f"Best Validation MAE: {best_val_mae}")
print(f"Best Hyperparameters: {best_hyperparams}")


Training bidirectional_rnn model with 64 units, learning rate 0.001, batch size 16
Validation MAE: 0.970288872718811
Training bidirectional_rnn model with 64 units, learning rate 0.001, batch size 32
Validation MAE: 1.5508111715316772
Training bidirectional_rnn model with 64 units, learning rate 0.001, batch size 64
Validation MAE: 1.2122737169265747
Training bidirectional_rnn model with 64 units, learning rate 0.0001, batch size 16
Validation MAE: 1.2876192331314087
Training bidirectional_rnn model with 64 units, learning rate 0.0001, batch size 32
Validation MAE: 1.6095991134643555
Training bidirectional_rnn model with 64 units, learning rate 0.0001, batch size 64
Validation MAE: 3.4364511966705322
Training bidirectional_rnn model with 128 units, learning rate 0.001, batch size 16
Validation MAE: 1.3526067733764648
Training bidirectional_rnn model with 128 units, learning rate 0.001, batch size 32
Validation MAE: 1.1391572952270508
Training bidirectional_rnn model with 128 units, lea

# Step 4: Evaluate Models

Based on the provided validation MAE scores and hyperparameters, we can compare the performance of the RNN, Bidirectional RNN, GRU, and LSTM models:

1. **RNN Model:**
   - Validation MAE: 1.1768
   - Hyperparameters: {'rnn_units': 256, 'learning_rate': 0.001, 'batch_size': 64}

2. **Bidirectional RNN Model:**
   - Validation MAE: 0.9703
   - Hyperparameters: {'rnn_units': 64, 'learning_rate': 0.001, 'batch_size': 16}

3. **GRU Model:**
   - Validation MAE: 0.5547
   - Hyperparameters: {'rnn_units': 128, 'learning_rate': 0.001, 'batch_size': 16}

4. **LSTM Model:**
   - Validation MAE: 0.2913
   - Hyperparameters: {'rnn_units': 64, 'learning_rate': 0.001, 'batch_size': 16}

**Comparison:**
- The LSTM model achieved the lowest validation MAE, indicating the best performance among the four models.
- The Bidirectional RNN model performed better than the RNN model but not as well as the GRU and LSTM models.
- The GRU model outperformed both the RNN and Bidirectional RNN models but had a slightly higher validation MAE compared to the LSTM model.
- Overall, the LSTM model with 64 units, a learning rate of 0.001, and a batch size of 16 appears to be the best choice based on the provided validation results and hyperparameters.